### Code used for reading and converting raster files to array


#### This code provide an automatic read of Bilraster files into array, and then organize the data in a time series to be used for further analysis.

#### The code will read a total of Z .map files, and will reshape each file from a (X, Y) to a (1, X * Y), and will assing each file matrix to a different line. Therefore we will have a time-series matrix of (Z, X * Y) shape. 

Developed by: Thiago Victor Medeiros do Nascimento

In [35]:
from pcraster import *
import numpy as np
from osgeo import gdal, gdalconst
from osgeo import gdal_array
from osgeo import osr
import matplotlib.pylab as plt
import subprocess
import glob,os
import time
import rasterio
import tqdm
import pandas as pd

import standard_precip
from standard_precip import spi
import warnings
warnings.filterwarnings('ignore')

Firstly we pre-read our dataset:

#### (a) Data lecture:

In [36]:
path =r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\dnCHIRPS'
filenames = glob.glob(path + "/*.bil")
print("The total number of files is:", len(filenames))

The total number of files is: 468


Firstly we take a look on the dataset:

In [37]:
mapfile = filenames[0]

RasterLayer = rasterio.open(mapfile)

ncols = RasterLayer.width
nrows = RasterLayer.height

numtotal = nrows*ncols

print("The total number of grids in the dataset is:", numtotal)

The total number of grids in the dataset is: 16330


We create one array to be filled:

In [38]:
precmongrids = np.zeros((len(filenames),numtotal),dtype=np.float32)
precmongrids.shape

(468, 16330)

Now we proceed with the data lecture and organization:

In [39]:
RasterLayer = rasterio.open(mapfile)
mapreadarray = RasterLayer.read()[0,:,:]
mapreadarrayres = np.reshape(mapreadarray, (1, numtotal))
mapreadarrayres

array([[275, 279, 271, ..., 417, 403, 382]], dtype=int16)

At this point, we use a for to go through each raster in our dataset (each raster represents a month), and organize the individual matrix in a unique dataset where each line will represent one month, and each row will represent one grid:

In [41]:
start = time.time()
i = 0
for mapfile in tqdm.tqdm(filenames):
       
    RasterLayer = rasterio.open(mapfile)
    mapreadarray = RasterLayer.read()[0,:,:]
    mapreadarrayres = np.reshape(mapreadarray, (1, numtotal))
    precmongrids[i,:] = mapreadarrayres
    
    i = i + 1
end = time.time()
print(end - start)

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:05<00:00, 80.75it/s]

5.799858093261719


At this point we can just check the shape of our dataset. It should have 468 monhts and 16330 grids:

In [42]:
precmongrids.shape

(468, 16330)

Now we create a dataframe with our organized precipitation data:

In [44]:
precmongridsdf = pd.DataFrame(index = pd.date_range(start='1-1981', end='1-2020', freq='M') , data = precmongrids)
precmongridsdf

,0,1,2,3,4,5,6,7,8,9,...,16320,16321,16322,16323,16324,16325,16326,16327,16328,16329
1981-01-31,275.0,279.0,271.0,267.0,267.0,254.0,248.0,242.0,241.0,242.0,...,388.0,406.0,392.0,379.0,381.0,421.0,417.0,417.0,403.0,382.0
1981-02-28,348.0,330.0,347.0,341.0,314.0,290.0,291.0,288.0,301.0,278.0,...,279.0,292.0,307.0,296.0,297.0,302.0,299.0,298.0,299.0,302.0
1981-03-31,301.0,310.0,313.0,316.0,325.0,328.0,324.0,324.0,328.0,320.0,...,306.0,301.0,303.0,303.0,307.0,319.0,322.0,327.0,328.0,330.0
1981-04-30,193.0,174.0,199.0,198.0,224.0,232.0,226.0,229.0,204.0,222.0,...,96.0,110.0,90.0,90.0,90.0,93.0,98.0,98.0,117.0,138.0
1981-05-31,183.0,196.0,208.0,198.0,176.0,177.0,177.0,174.0,162.0,157.0,...,11.0,11.0,11.0,13.0,13.0,10.0,7.0,7.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,35.0,38.0,34.0,32.0,32.0,31.0,30.0,32.0,30.0,31.0,...,32.0,30.0,29.0,30.0,29.0,28.0,27.0,27.0,29.0,32.0
2019-09-30,140.0,135.0,140.0,131.0,143.0,142.0,123.0,129.0,128.0,117.0,...,58.0,56.0,55.0,56.0,54.0,55.0,58.0,58.0,60.0,63.0
2019-10-31,225.0,217.0,220.0,230.0,231.0,233.0,228.0,214.0,216.0,218.0,...,163.0,164.0,162.0,165.0,166.0,163.0,168.0,170.0,176.0,174.0
2019-11-30,220.0,210.0,215.0,213.0,224.0,223.0,220.0,214.0,219.0,228.0,...,123.0,118.0,115.0,125.0,129.0,144.0,147.0,145.0,153.0,158.0


Now we export our dataset to a .CSV to be easily opened afterwards:

In [15]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\precmongrids.csv', precmongrids, delimiter=',')

If we want to save our dataframe in a .XLSX file:

In [16]:
precmongridsdf.to_excel(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\precmongrids.xlsx')

#### (b) SPI computation for each grid:

First we read the dataset of precipitation that was already organized in part (a):

In [47]:
data = pd.read_csv(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\precmongrids.csv')
data

,date,0,1,2,3,4,5,6,7,8,...,16320,16321,16322,16323,16324,16325,16326,16327,16328,16329
0,31/01/1981,275.0,279.0,271.0,267.0,267.0,254.0,248.0,242.0,241.0,...,388.0,406.0,392.0,379.0,381.0,421.0,417.0,417.0,403.0,382.0
1,28/02/1981,348.0,330.0,347.0,341.0,314.0,290.0,291.0,288.0,301.0,...,279.0,292.0,307.0,296.0,297.0,302.0,299.0,298.0,299.0,302.0
2,31/03/1981,301.0,310.0,313.0,316.0,325.0,328.0,324.0,324.0,328.0,...,306.0,301.0,303.0,303.0,307.0,319.0,322.0,327.0,328.0,330.0
3,30/04/1981,193.0,174.0,199.0,198.0,224.0,232.0,226.0,229.0,204.0,...,96.0,110.0,90.0,90.0,90.0,93.0,98.0,98.0,117.0,138.0
4,31/05/1981,183.0,196.0,208.0,198.0,176.0,177.0,177.0,174.0,162.0,...,11.0,11.0,11.0,13.0,13.0,10.0,7.0,7.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,31/08/2019,35.0,38.0,34.0,32.0,32.0,31.0,30.0,32.0,30.0,...,32.0,30.0,29.0,30.0,29.0,28.0,27.0,27.0,29.0,32.0
464,30/09/2019,140.0,135.0,140.0,131.0,143.0,142.0,123.0,129.0,128.0,...,58.0,56.0,55.0,56.0,54.0,55.0,58.0,58.0,60.0,63.0
465,31/10/2019,225.0,217.0,220.0,230.0,231.0,233.0,228.0,214.0,216.0,...,163.0,164.0,162.0,165.0,166.0,163.0,168.0,170.0,176.0,174.0
466,30/11/2019,220.0,210.0,215.0,213.0,224.0,223.0,220.0,214.0,219.0,...,123.0,118.0,115.0,125.0,129.0,144.0,147.0,145.0,153.0,158.0


At this step we can check the number pof grids in our dataset:

In [48]:
numgrids = data.shape[1] - 1
numgrids

16330

Before computing the SPIs, we create empty arrays for stack the results of SPI computation for each loop:

In [52]:
spi1 = np.empty([len(filenames), ])
spi3 = np.empty([len(filenames), ])
spi6 = np.empty([len(filenames), ])
spi9 = np.empty([len(filenames), ])
spi12 = np.empty([len(filenames), ])
spi24 = np.empty([len(filenames), ])
spi48 = np.empty([len(filenames), ])

First we set the SPI function calling it as spi_rain:

In [45]:
spi_rain = spi.SPI()

Now we will do a for for each grid in our dataset, and therefore compute the required SPis using the function "spi_rain":

In [53]:
start = time.time()
    
for grid in tqdm.tqdm(range(numgrids)):
    
    data2 = data.loc[:, ["date", str(grid)]]

    # SPI-1
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=1, fit_type="lmom", dist_type="gam")    
    spi1 = np.vstack((spi1, aux.iloc[:,2].values))
    
    # SPI-3
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=3, fit_type="lmom", dist_type="gam")    
    spi3 = np.vstack((spi3, aux.iloc[:,2].values))
    
    # SPI-6
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=6, fit_type="lmom", dist_type="gam")    
    spi6 = np.vstack((spi6, aux.iloc[:,2].values))  
    
    # SPI-9
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=9, fit_type="lmom", dist_type="gam")    
    spi9 = np.vstack((spi9, aux.iloc[:,2].values))
    
    # SPI-12
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=12, fit_type="lmom", dist_type="gam")    
    spi12 = np.vstack((spi12, aux.iloc[:,2].values))
    
    # SPI-24
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=24, fit_type="lmom", dist_type="gam")    
    spi24 = np.vstack((spi24, aux.iloc[:,2].values))

    # SPI-48
    aux = spi_rain.calculate(data2, 'date', str(grid), freq="M", scale=48, fit_type="lmom", dist_type="gam")    
    spi48 = np.vstack((spi48, aux.iloc[:,2].values))    
    
end = time.time()
print(end - start)

100%|██████████████████████████████████████████████████████████████████████████| 16330/16330 [2:03:48<00:00,  2.20it/s]

7428.209497928619


Now we delete the first row and traspose the results:

In [54]:
spi1 = np.delete(spi1, [0], axis = 0).T
spi3 = np.delete(spi3, [0], axis = 0).T
spi6 = np.delete(spi6, [0], axis = 0).T
spi9 = np.delete(spi9, [0], axis = 0).T
spi12 = np.delete(spi12, [0], axis = 0).T
spi24 = np.delete(spi24, [0], axis = 0).T
spi48 = np.delete(spi48, [0], axis = 0).T

In [62]:
spi1

array([[-0.63202762, -0.57175492, -0.60991597, ...,  1.37649818,
         1.27751699,  1.0897184 ],
       [ 0.52873685,  0.26382294,  0.44129962, ...,  0.19205436,
         0.26004079,  0.28865001],
       [-0.01441963,  0.10325619,  0.09588086, ...,  0.52841591,
         0.534137  ,  0.51598494],
       ...,
       [ 1.41390261,  1.27712162,  1.40425307, ...,  0.27414199,
         0.26501261,  0.1717614 ],
       [-0.02093242, -0.2164415 , -0.07527487, ..., -1.4554596 ,
        -1.20925809, -1.28184177],
       [-0.51609831, -0.37143233, -0.40075027, ..., -1.08493912,
        -1.04451341, -0.96729763]])

Now we save the SPI results:

In [56]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi1.csv', spi1, delimiter=',')

In [57]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi3.csv', spi3, delimiter=',')

In [58]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi6.csv', spi6, delimiter=',')

In [59]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi9.csv', spi9, delimiter=',')

In [60]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi12.csv', spi12, delimiter=',')

In [61]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi24.csv', spi24, delimiter=',')

In [ ]:
np.savetxt(r'C:\Users\User\OneDrive\IST\RESEARCH\5_SPI\rondonia\spi48.csv', spi48, delimiter=',')